# Azure AI Foundry Environment Setup 

This notebook will guide you through setting up your environment for the Azure AI Foundry.

## Prerequisites
- Python 3.8 or later
- Azure subscription with AI services access
- Basic Python knowledge

In [1]:
# Install required packages (if not already installed):
!pip install azure-identity azure-ai-projects

## Azure Authentication Setup
First, we'll verify our Azure credentials and setup.

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
import os

# Initialize Azure credentials
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

✓ Successfully initialized DefaultAzureCredential


## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Create `cred.json` file
> 2. Update the project connection string in your `cred.json` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## Understanding AIProjectClient

The AIProjectClient is a key component for interacting with Azure AI services that:

- **Manages Connections**: Lists and accesses Azure AI resources like OpenAI models
- **Handles Authentication**: Securely connects using Azure credentials  
- **Enables Model Access**: Provides interfaces to use AI models and deployments
- **Manages Project Settings**: Controls configurations for your Azure AI project

The client requires:
- A project connection string (from Azure AI project settings)
- Valid Azure credentials

You can find your project connection string in Azure AI Studio under Project Settings:



In [3]:
import os
import json


def find_file(filename, search_path):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# Search in the parent directory and its subdirectories
parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
file_path = find_file('cred.json', parent_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)
    
    # Print the values to verify
    print("Project Connection String:", loaded_config['PROJECT_CONNECTION_STRING'])
    print("Tenant ID:", loaded_config['TENANT_ID'])
    
except FileNotFoundError:
    print(f"Could not find file at: {file_path}")
except json.JSONDecodeError:
    print(f"File exists but contains invalid JSON")


Found cred.json at: /workspaces/codespaces-blank/Azure-AI-Foundry/cred.json
Project Connection String: eastus2.api.azureml.ms;1c2fd79b-ad21-4ad0-8d53-12de16650452;rg-sarath-7958_ai;sarath-0309
Tenant ID: 02e58275-def8-41c4-82c4-f7864c28f7c9


In [4]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')
print(loaded_config["PROJECT_CONNECTION_STRING"])

# Initialize AIProjectClient with connection string and credentials
try:
    client = AIProjectClient.from_connection_string(
        conn_str=loaded_config["PROJECT_CONNECTION_STRING"],
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

eastus2.api.azureml.ms;1c2fd79b-ad21-4ad0-8d53-12de16650452;rg-sarath-7958_ai;sarath-0309
✓ Successfully initialized AIProjectClient


## Verify Access to Models and Connections

We need to validate:
- GPT models (gpt-4o, gpt-4o-mini) for chat/completion

This validation ensures we have all the components needed to build our AI applications.

In [8]:
# List the properties of all connections
connections = client.connections.list()
print(f"====> Listing of all connections (found {len(connections)}):")
for connection in connections:
    print(connection)

# List the properties of all connections of a particular "type" (in this sample, Azure OpenAI connections)
connections = client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)
print(f"====> Listing of all Azure Open AI connections (found {len(connections)}):")
for connection in connections:
    print(connection)

# Get the properties of the default connection of a particular "type", with credentials
connection = client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SERVICES,
    include_credentials=True,  # Optional. Defaults to "False"
)
print("====> Get default Azure AI Services connection:")
print(connection)

print("====> Get connection by name:")
print(connection)

====> Listing of all connections (found 4):
{
 "name": "ai-sarath9175ai752558758647_aoai",
 "id": "/subscriptions/1c2fd79b-ad21-4ad0-8d53-12de16650452/resourceGroups/rg-sarath-7958_ai/providers/Microsoft.MachineLearningServices/workspaces/sarath-0309/connections/ai-sarath9175ai752558758647_aoai",
 "authentication_type": "ApiKey",
 "connection_type": "ConnectionType.AZURE_OPEN_AI",
 "endpoint_url": "https://ai-sarath9175ai752558758647.openai.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "ai-sarath9175ai752558758647",
 "id": "/subscriptions/1c2fd79b-ad21-4ad0-8d53-12de16650452/resourceGroups/rg-sarath-7958_ai/providers/Microsoft.MachineLearningServices/workspaces/sarath-0309/connections/ai-sarath9175ai752558758647",
 "authentication_type": "ApiKey",
 "connection_type": "ConnectionType.AZURE_AI_SERVICES",
 "endpoint_url": "https://ai-sarath9175ai752558758647.cognitiveservices.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "sarath-0309/workspaceblobstor